# Recommendation System

In [1]:
from collections import Counter
from itertools import combinations
import re

import pandas as pd
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity

import matplotlib.pyplot as plt
import seaborn as sns

import pickle as pk

from jupyterthemes import jtplot
jtplot.style(figsize=(15, 9))

## Data

In [2]:
movies = pd.read_csv('data/movies_dataset_CLEAN_noNaN.csv', index_col='id')

# Vectors
vectors_keywords = pd.read_csv('data/keywords_vectors.csv', index_col=0)
vectors_cast     = pd.read_csv('data/cast_vectors.csv', index_col=0)
vectors_crew     = pd.read_csv('data/crew_vectors.csv', index_col=0)
vectors_genres   = pd.read_csv('data/genres_vectors.csv', index_col=0)

# Sum of Vectors
sum_vectors_keywords = pd.read_csv('data/keywords_sum_vectors.csv', index_col='id')
sum_vectors_cast     = pd.read_csv('data/cast_sum_vectors.csv', index_col='id')
sum_vectors_crew     = pd.read_csv('data/crew_sum_vectors.csv', index_col='id')

# Genres cosine similarity
genres_cs = pd.read_csv('DRAFT/data/cosine_similarity_genres.csv', index_col=0)

## Movies choice

In [3]:
like = [
    'From Dusk Till Dawn',
    'Alien',
    'The Descent',
]

In [4]:
like_df = movies[movies['title'].isin(like)]

In [5]:
like_df

,genre_ids,overview,popularity,release_date,title,vote_average,vote_count,genres,cast,crew,keywords,label,label_encoder
id,,,,,,,,,,,,,
348,"[27, 878]","During its return to the earth, commercial spa...",62.215,1979-05-25,Alien,8.1,12914,"Horror, Science Fiction","[{'cast_id': 10205, 'name': 'Sigourney Weaver'...","[{'crew_id': 578, 'name': 'Ridley Scott', 'dep...","[{'id': 803, 'name': 'android'}, {'id': 1612, ...",Science Fiction,14
755,"[27, 28, 53, 80]",Seth Gecko and his younger brother Richard are...,27.675,1996-01-19,From Dusk Till Dawn,7.0,5301,"Horror, Action, Thriller, Crime","[{'cast_id': 1461, 'name': 'George Clooney', '...","[{'crew_id': 2294, 'name': 'Robert Rodriguez',...","[{'id': 246, 'name': 'dancing'}, {'id': 380, '...",Horror,10
9392,"[12, 27]","After a tragic accident, six friends reunite f...",22.417,2005-07-08,The Descent,7.0,3394,"Adventure, Horror","[{'cast_id': 57574, 'name': 'Shauna Macdonald'...","[{'crew_id': 558, 'name': 'David Julyan', 'dep...","[{'id': 1852, 'name': 'mutant'}, {'id': 1963, ...",Horror,10


In [106]:
class IWantToWatchSomethingLike:
    def __init__(self, i_like:list):
        self.like = i_like
        self.like_df = movies[movies['title'].isin(self.like)]
        
        self.cs_THRESHOLD = 0.95
        self.MIN_VOTE_COUNT = 1000
        self.MIN_POPULARITY = 14
        self.MIN_RATING = self.gather_like_info()

        self.similar_movies = self.get_similar_movies()
        self.cosine_similarity = {
            'keywords': pd.DataFrame([]),
            'cast'    : pd.DataFrame([]),
            'crew'    : pd.DataFrame([]),
        }
        self.get_cosine_similarity()
        
    def get_cosine_similarity(self):
        keywords_cs = cosine_similarity(
            sum_vectors_keywords.loc[self.like_df.index],
            sum_vectors_keywords.loc[self.similar_movies.index]
        )
        cast_cs = cosine_similarity(
            sum_vectors_cast.loc[self.like_df.index],
            sum_vectors_cast.loc[self.similar_movies.index]
        )
        crew_cs = cosine_similarity(
            sum_vectors_crew.loc[self.like_df.index],
            sum_vectors_crew.loc[self.similar_movies.index]
        )
        
        self.cosine_similarity['keywords'] = pd.DataFrame(
            index=self.similar_movies.index,
            columns=self.like_df['title'],
            data=keywords_cs.transpose()
        )
        
        self.cosine_similarity['cast'] = pd.DataFrame(
            index=self.similar_movies.index,
            columns=self.like_df['title'],
            data=keywords_cs.transpose()
        )
        
        self.cosine_similarity['crew'] = pd.DataFrame(
            index=self.similar_movies.index,
            columns=self.like_df['title'],
            data=keywords_cs.transpose()
        )
    
    def gather_inds_by_cs(self)->list:
        inds = []
        for title in self.like:
            inds += self.cosine_similarity['keywords'][
                self.cosine_similarity['keywords'][title]>self.cs_THRESHOLD
            ][title].index.to_list()\
            + self.cosine_similarity['cast'][
                self.cosine_similarity['cast'][title]>self.cs_THRESHOLD
            ][title].index.to_list()\
            + self.cosine_similarity['crew'][
                self.cosine_similarity['crew'][title]>self.cs_THRESHOLD
            ][title].index.to_list()
        
        return list(set(inds))
    
    def flatten_list(self, list_of_lists:list)->list:
        return [l for ll in list_of_lists for l in ll]
    
    # REWRITE
    def reduce_genres(self)->list:
        genres_list_of_lists = [gl.split(', ') for gl in self.like_df['genres'].to_list()]
        genres_flat_list = self.flatten_list(genres_list_of_lists)
        
        genres_list = set(genres_flat_list)
        genres_list = list(genres_list)
#         genres_list = list(genres_list - set(like_df['label'].to_list()))
        
        neariest_genres = []
        for g in genres_list:
            neariest_genres.append(genres_cs.loc[:, genres_list][genres_cs[g]>0.6].index.to_list())
        
        neariest_genres = self.flatten_list(neariest_genres)

        neariest_genres_counter = dict(Counter(neariest_genres))

        genres_more_than_1_apperance = [k for k, v in neariest_genres_counter.items() if v>1]
        
        genres_list.extend(genres_more_than_1_apperance)
        
        return list(set(genres_list))
    
    def show_recommendation(self, amount=10, sorted_by='vote_average', reduce_genres=False)->pd.DataFrame:
        inds = self.gather_inds_by_cs()
        recommendation = self.similar_movies.loc[inds]
        
        if reduce_genres:
            genres_list = self.reduce_genres()
            s = vectors_genres.loc[recommendation.index, genres_list].sum(axis=1)>0
            recommendation = recommendation.loc[s[s].index]

        if sorted_by in recommendation.columns and amount < recommendation.shape[0]:
            return recommendation.sort_values([sorted_by], ascending=False)[:amount]
        
        return recommendation
        
    def get_similar_movies(self)->pd.DataFrame:
        return movies[
                         (~movies['title'].isin(self.like))
                         & (movies['vote_average'] >= self.MIN_RATING)
                         & (movies['vote_count'] >= self.MIN_VOTE_COUNT)
                         & (movies['popularity'] > self.MIN_POPULARITY)
                         & (movies['label'].isin(self.like_df['label'].to_list()))
                     ]

    def gather_like_info(self)->float:
        return self.like_df['vote_average'].min()-0.2
    
    def show_what_i_like(self)->pd.DataFrame:
        return self.like_df
    
    

In [102]:
what_to_watch = IWantToWatchSomethingLike(like)

what_to_watch.show_what_i_like()

,genre_ids,overview,popularity,release_date,title,vote_average,vote_count,genres,cast,crew,keywords,label,label_encoder
id,,,,,,,,,,,,,
348,"[27, 878]","During its return to the earth, commercial spa...",62.215,1979-05-25,Alien,8.1,12914,"Horror, Science Fiction","[{'cast_id': 10205, 'name': 'Sigourney Weaver'...","[{'crew_id': 578, 'name': 'Ridley Scott', 'dep...","[{'id': 803, 'name': 'android'}, {'id': 1612, ...",Science Fiction,14
755,"[27, 28, 53, 80]",Seth Gecko and his younger brother Richard are...,27.675,1996-01-19,From Dusk Till Dawn,7.0,5301,"Horror, Action, Thriller, Crime","[{'cast_id': 1461, 'name': 'George Clooney', '...","[{'crew_id': 2294, 'name': 'Robert Rodriguez',...","[{'id': 246, 'name': 'dancing'}, {'id': 380, '...",Horror,10
9392,"[12, 27]","After a tragic accident, six friends reunite f...",22.417,2005-07-08,The Descent,7.0,3394,"Adventure, Horror","[{'cast_id': 57574, 'name': 'Shauna Macdonald'...","[{'crew_id': 558, 'name': 'David Julyan', 'dep...","[{'id': 1852, 'name': 'mutant'}, {'id': 1963, ...",Horror,10


In [24]:
what_to_watch.show_recommendation(sorted_by='release_date')

,genre_ids,overview,popularity,release_date,title,vote_average,vote_count,genres,cast,crew,keywords,label,label_encoder
id,,,,,,,,,,,,,
713704,"[53, 27]",A reunion between two estranged sisters gets c...,754.930,2023-04-12,Evil Dead Rise,7.0,1765,"Thriller, Horror","[{'cast_id': 1186659, 'name': 'Lily Sullivan',...","[{'crew_id': 7623, 'name': 'Sam Raimi', 'depar...","[{'id': 380, 'name': 'sibling relationship'}, ...",Horror,10
934433,"[27, 9648, 53]","Following the latest Ghostface killings, the f...",315.504,2023-03-08,Scream VI,7.2,1418,"Horror, Mystery, Thriller","[{'cast_id': 1373659, 'name': 'Melissa Barrera...","[{'crew_id': 409, 'name': 'Cathy Konrad', 'dep...","[{'id': 242, 'name': 'new york city'}, {'id': ...",Horror,10
76600,"[878, 12, 28]",Set more than a decade after the events of the...,899.251,2022-12-14,Avatar: The Way of Water,7.7,8861,"Science Fiction, Adventure, Action","[{'cast_id': 65731, 'name': 'Sam Worthington',...","[{'crew_id': 1721, 'name': 'Stephen E. Rivkin'...","[{'id': 697, 'name': 'loss of loved one'}, {'i...",Science Fiction,14
663712,"[27, 53]","After being resurrected by a sinister entity, ...",197.173,2022-10-06,Terrifier 2,6.9,1268,"Horror, Thriller","[{'cast_id': 1882502, 'name': 'Lauren LaVera',...","[{'crew_id': 83331, 'name': 'Steven Barton', '...","[{'id': 3199, 'name': 'clown'}, {'id': 3335, '...",Horror,10
913290,"[27, 9648, 53]","In town for a job interview, a young woman arr...",40.927,2022-09-08,Barbarian,6.9,1530,"Horror, Mystery, Thriller","[{'cast_id': 1335315, 'name': 'Georgina Campbe...","[{'crew_id': 376, 'name': 'Arnon Milchan', 'de...","[{'id': 739, 'name': 'sexual abuse'}, {'id': 1...",Horror,10
756999,"[27, 53]","Finney Blake, a shy but clever 13-year-old boy...",77.743,2022-06-22,The Black Phone,7.7,3946,"Horror, Thriller","[{'cast_id': 2803710, 'name': 'Mason Thames', ...","[{'crew_id': 494, 'name': 'Terri Taylor', 'dep...","[{'id': 380, 'name': 'sibling relationship'}, ...",Horror,10
696806,"[12, 878]","After accidentally crash-landing in 2022, time...",54.317,2022-03-11,The Adam Project,7.1,3652,"Adventure, Science Fiction","[{'cast_id': 10859, 'name': 'Ryan Reynolds', '...","[{'crew_id': 10956, 'name': 'Josh McLaglen', '...","[{'id': 4379, 'name': 'time travel'}, {'id': 1...",Science Fiction,14
522402,"[878, 18, 12]","On a post-apocalyptic Earth, a robot, built to...",57.570,2021-11-04,Finch,7.9,2945,"Science Fiction, Drama, Adventure","[{'cast_id': 31, 'name': 'Tom Hanks', 'charact...","[{'crew_id': 24, 'name': 'Robert Zemeckis', 'd...","[{'id': 14544, 'name': 'robot'}, {'id': 189092...",Science Fiction,14
576845,"[27, 9648, 53]","A young girl, passionate about fashion design,...",33.303,2021-10-21,Last Night in Soho,7.4,2608,"Horror, Mystery, Thriller","[{'cast_id': 1356758, 'name': 'Thomasin McKenz...","[{'crew_id': 2236, 'name': 'Tim Bevan', 'depar...","[{'id': 212, 'name': 'london, england'}, {'id'...",Horror,10


In [61]:
what_to_watch.show_recommendation(amount=10000)

,genre_ids,overview,popularity,release_date,title,vote_average,vote_count,genres,cast,crew,keywords,label,label_encoder
id,,,,,,,,,,,,,
1547,"[27, 35, 53]",A mother and her two teenage sons move to a se...,20.128,1987-07-31,The Lost Boys,7.1,1732,"Horror, Comedy, Thriller","[{'cast_id': 12261, 'name': 'Jason Patric', 'c...","[{'crew_id': 153, 'name': 'Thomas Newman', 'de...","[{'id': 387, 'name': 'california'}, {'id': 542...",Horror,10
17431,"[878, 18]",With only three weeks left in his three year c...,19.520,2009-06-12,Moon,7.6,5130,"Science Fiction, Drama","[{'cast_id': 6807, 'name': 'Sam Rockwell', 'ch...","[{'crew_id': 5497, 'name': 'Aleksandar Jovanov...","[{'id': 305, 'name': 'moon'}, {'id': 310, 'nam...",Science Fiction,14
338967,"[35, 27]","Columbus, Tallahassee, Wichita, and Little Roc...",39.657,2019-10-09,Zombieland: Double Tap,7.0,5062,"Comedy, Horror","[{'cast_id': 57755, 'name': 'Woody Harrelson',...","[{'crew_id': 561, 'name': 'John Papsidera', 'd...","[{'id': 833, 'name': 'the white house'}, {'id'...",Horror,10
10779,"[27, 35]","Once an architect, Frank Bannister now passes ...",19.185,1996-07-18,The Frighteners,6.9,1323,"Horror, Comedy","[{'cast_id': 521, 'name': 'Michael J. Fox', 'c...","[{'crew_id': 24, 'name': 'Robert Zemeckis', 'd...","[{'id': 456, 'name': 'mother'}, {'id': 798, 'n...",Horror,10
19995,"[28, 12, 14, 878]","In the 22nd century, a paraplegic Marine is di...",209.916,2009-12-15,Avatar,7.6,29311,"Action, Adventure, Fantasy, Science Fiction","[{'cast_id': 65731, 'name': 'Sam Worthington',...","[{'crew_id': 1721, 'name': 'Stephen E. Rivkin'...","[{'id': 1463, 'name': 'culture clash'}, {'id':...",Science Fiction,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19908,"[35, 27]",Columbus has made a habit of running from what...,39.697,2009-10-07,Zombieland,7.3,11268,"Comedy, Horror","[{'cast_id': 57755, 'name': 'Woody Harrelson',...","[{'crew_id': 561, 'name': 'John Papsidera', 'd...","[{'id': 291, 'name': 'circus'}, {'id': 380, 'n...",Horror,10
336843,"[878, 28, 12, 53]",Thomas leads his group of escaped Gladers on t...,87.886,2018-01-10,Maze Runner: The Death Cure,7.1,6989,"Science Fiction, Action, Adventure, Thriller","[{'cast_id': 527393, 'name': ""Dylan O'Brien"", ...","[{'crew_id': 2215, 'name': 'Denise Chamian', '...","[{'id': 818, 'name': 'based on novel or book'}...",Science Fiction,14
985,"[14, 27]",First time father Henry Spencer tries to survi...,16.080,1977-03-19,Eraserhead,7.4,2131,"Fantasy, Horror","[{'cast_id': 6718, 'name': 'Jack Nance', 'char...","[{'crew_id': 4434, 'name': 'Frederick Elmes', ...","[{'id': 1009, 'name': 'baby'}, {'id': 1852, 'n...",Horror,10


In [60]:
what_to_watch.show_recommendation(amount=10000, reduce_genres=True)

,genre_ids,overview,popularity,release_date,title,vote_average,vote_count,genres,cast,crew,keywords,label,label_encoder
id,,,,,,,,,,,,,
1547,"[27, 35, 53]",A mother and her two teenage sons move to a se...,20.128,1987-07-31,The Lost Boys,7.1,1732,"Horror, Comedy, Thriller","[{'cast_id': 12261, 'name': 'Jason Patric', 'c...","[{'crew_id': 153, 'name': 'Thomas Newman', 'de...","[{'id': 387, 'name': 'california'}, {'id': 542...",Horror,10
17431,"[878, 18]",With only three weeks left in his three year c...,19.520,2009-06-12,Moon,7.6,5130,"Science Fiction, Drama","[{'cast_id': 6807, 'name': 'Sam Rockwell', 'ch...","[{'crew_id': 5497, 'name': 'Aleksandar Jovanov...","[{'id': 305, 'name': 'moon'}, {'id': 310, 'nam...",Science Fiction,14
338967,"[35, 27]","Columbus, Tallahassee, Wichita, and Little Roc...",39.657,2019-10-09,Zombieland: Double Tap,7.0,5062,"Comedy, Horror","[{'cast_id': 57755, 'name': 'Woody Harrelson',...","[{'crew_id': 561, 'name': 'John Papsidera', 'd...","[{'id': 833, 'name': 'the white house'}, {'id'...",Horror,10
10779,"[27, 35]","Once an architect, Frank Bannister now passes ...",19.185,1996-07-18,The Frighteners,6.9,1323,"Horror, Comedy","[{'cast_id': 521, 'name': 'Michael J. Fox', 'c...","[{'crew_id': 24, 'name': 'Robert Zemeckis', 'd...","[{'id': 456, 'name': 'mother'}, {'id': 798, 'n...",Horror,10
19995,"[28, 12, 14, 878]","In the 22nd century, a paraplegic Marine is di...",209.916,2009-12-15,Avatar,7.6,29311,"Action, Adventure, Fantasy, Science Fiction","[{'cast_id': 65731, 'name': 'Sam Worthington',...","[{'crew_id': 1721, 'name': 'Stephen E. Rivkin'...","[{'id': 1463, 'name': 'culture clash'}, {'id':...",Science Fiction,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19908,"[35, 27]",Columbus has made a habit of running from what...,39.697,2009-10-07,Zombieland,7.3,11268,"Comedy, Horror","[{'cast_id': 57755, 'name': 'Woody Harrelson',...","[{'crew_id': 561, 'name': 'John Papsidera', 'd...","[{'id': 291, 'name': 'circus'}, {'id': 380, 'n...",Horror,10
336843,"[878, 28, 12, 53]",Thomas leads his group of escaped Gladers on t...,87.886,2018-01-10,Maze Runner: The Death Cure,7.1,6989,"Science Fiction, Action, Adventure, Thriller","[{'cast_id': 527393, 'name': ""Dylan O'Brien"", ...","[{'crew_id': 2215, 'name': 'Denise Chamian', '...","[{'id': 818, 'name': 'based on novel or book'}...",Science Fiction,14
985,"[14, 27]",First time father Henry Spencer tries to survi...,16.080,1977-03-19,Eraserhead,7.4,2131,"Fantasy, Horror","[{'cast_id': 6718, 'name': 'Jack Nance', 'char...","[{'crew_id': 4434, 'name': 'Frederick Elmes', ...","[{'id': 1009, 'name': 'baby'}, {'id': 1852, 'n...",Horror,10


In [92]:
def show_diff_in_recommendation(df):
    no_reduced_genres = set(df.show_recommendation(amount=10000).index.to_list())
    reduced_genres = set(df.show_recommendation(amount=10000, reduce_genres=True).index.to_list())
    return movies.loc[list(no_reduced_genres - reduced_genres)]

In [103]:
show_diff_in_recommendation(what_to_watch)

,genre_ids,overview,popularity,release_date,title,vote_average,vote_count,genres,cast,crew,keywords,label,label_encoder
id,,,,,,,,,,,,,


In [104]:
w = IWantToWatchSomethingLike(['Apocalypse Now', 'Saving Private Ryan'])

In [89]:
w.show_recommendation(amount=10000)

,genre_ids,overview,popularity,release_date,title,vote_average,vote_count,genres,cast,crew,keywords,label,label_encoder
id,,,,,,,,,,,,,
769,"[18, 80]","The true story of Henry Hill, a half-Irish, ha...",48.506,1990-09-12,GoodFellas,8.5,11372,"Drama, Crime","[{'cast_id': 11477, 'name': 'Ray Liotta', 'cha...","[{'crew_id': 1032, 'name': 'Martin Scorsese', ...","[{'id': 242, 'name': 'new york city'}, {'id': ...",Drama,6
770,"[18, 10752, 10749]",The spoiled daughter of a Georgia plantation o...,31.041,1939-12-15,Gone with the Wind,8.0,3603,"Drama, War, Romance","[{'cast_id': 10538, 'name': 'Vivien Leigh', 'c...","[{'crew_id': 2774, 'name': 'Ernest Haller', 'd...","[{'id': 393, 'name': 'civil war'}, {'id': 697,...",Drama,6
278,"[18, 80]",Framed in the 1940s for the double murder of h...,107.982,1994-09-23,The Shawshank Redemption,8.7,24021,"Drama, Crime","[{'cast_id': 504, 'name': 'Tim Robbins', 'char...","[{'crew_id': 153, 'name': 'Thomas Newman', 'de...","[{'id': 378, 'name': 'prison'}, {'id': 417, 'n...",Drama,6
289,"[18, 10749]","In Casablanca, Morocco in December 1941, a cyn...",27.961,1942-11-26,Casablanca,8.2,4783,"Drama, Romance","[{'cast_id': 4110, 'name': 'Humphrey Bogart', ...","[{'crew_id': 2005, 'name': 'Arthur Edeson', 'd...","[{'id': 128, 'name': 'love triangle'}, {'id': ...",Drama,6
423,"[18, 10752]",The true story of pianist Władysław Szpilman's...,35.898,2002-09-17,The Pianist,8.4,8143,"Drama, War","[{'cast_id': 3490, 'name': 'Adrien Brody', 'ch...","[{'crew_id': 970, 'name': 'Celestia Fox', 'dep...","[{'id': 836, 'name': 'resistance'}, {'id': 173...",Drama,6
424,"[18, 36, 10752]",The true story of how businessman Oskar Schind...,60.506,1993-12-15,Schindler's List,8.6,14198,"Drama, History, War","[{'cast_id': 3896, 'name': 'Liam Neeson', 'cha...","[{'crew_id': 491, 'name': 'John Williams', 'de...","[{'id': 818, 'name': 'based on novel or book'}...",History,9
7345,[18],"Ruthless silver miner, turned oil prospector, ...",39.058,2007-12-26,There Will Be Blood,8.1,5771,Drama,"[{'cast_id': 11856, 'name': 'Daniel Day-Lewis'...","[{'crew_id': 1809, 'name': 'Dylan Tichenor', '...","[{'id': 380, 'name': 'sibling relationship'}, ...",Drama,6
324786,"[18, 36, 10752]","WWII American Army Medic Desmond T. Doss, who ...",145.130,2016-10-07,Hacksaw Ridge,8.2,12366,"Drama, History, War","[{'cast_id': 37625, 'name': 'Andrew Garfield',...","[{'crew_id': 2545, 'name': 'Lawrence Bender', ...","[{'id': 233, 'name': 'japan'}, {'id': 1956, 'n...",History,9
947,"[12, 18, 36, 10752]",The story of British officer T.E. Lawrence's m...,25.962,1962-12-11,Lawrence of Arabia,8.0,2606,"Adventure, Drama, History, War","[{'cast_id': 11390, 'name': ""Peter O'Toole"", '...","[{'crew_id': 244, 'name': 'Ray Lovejoy', 'depa...","[{'id': 487, 'name': 'cairo'}, {'id': 1975, 'n...",History,9


In [86]:
movies[movies['title'].isin(['Apocalypse Now', 'Saving Private Ryan'])]

,genre_ids,overview,popularity,release_date,title,vote_average,vote_count,genres,cast,crew,keywords,label,label_encoder
id,,,,,,,,,,,,,
857,"[18, 36, 10752]","As U.S. troops storm the beaches of Normandy, ...",57.916,1998-07-24,Saving Private Ryan,8.2,14325,"Drama, History, War","[{'cast_id': 31, 'name': 'Tom Hanks', 'charact...","[{'crew_id': 491, 'name': 'John Williams', 'de...","[{'id': 1327, 'name': 'war crimes'}, {'id': 14...",History,9
28,"[18, 10752]","At the height of the Vietnam war, Captain Benj...",30.775,1979-08-15,Apocalypse Now,8.3,7290,"Drama, War","[{'cast_id': 3084, 'name': 'Marlon Brando', 'c...","[{'crew_id': 154, 'name': 'Walter Murch', 'dep...","[{'id': 591, 'name': 'cia'}, {'id': 726, 'name...",Drama,6


In [88]:
movies[movies['title'].isin(['Platoon', 'Full Metal Jacket'])]

,genre_ids,overview,popularity,release_date,title,vote_average,vote_count,genres,cast,crew,keywords,label,label_encoder
id,,,,,,,,,,,,,
600,"[18, 10752]",A pragmatic U.S. Marine observes the dehumaniz...,41.845,1987-06-26,Full Metal Jacket,8.1,9455,"Drama, War","[{'cast_id': 8654, 'name': 'Matthew Modine', '...","[{'crew_id': 240, 'name': 'Stanley Kubrick', '...","[{'id': 236, 'name': 'suicide'}, {'id': 549, '...",Drama,6
792,"[18, 10752, 28]","As a young and naive recruit in Vietnam, Chris...",23.723,1986-12-19,Platoon,7.7,3980,"Drama, War, Action","[{'cast_id': 13022, 'name': 'Tom Berenger', 'c...","[{'crew_id': 149, 'name': 'Robert Richardson',...","[{'id': 502, 'name': 'ambush'}, {'id': 1327, '...",Drama,6


In [105]:
show_diff_in_recommendation(w)

,genre_ids,overview,popularity,release_date,title,vote_average,vote_count,genres,cast,crew,keywords,label,label_encoder
id,,,,,,,,,,,,,
